In [1]:
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
from tqdm import tqdm_notebook as tqdm

import matplotlib.pylab as plt
import statsmodels.api as sm
import scipy.stats as st
import seaborn as sns
import pandas as pd
import numpy as np
import librosa
import os

считываем таблицу:

In [2]:
dataframe = pd.read_csv("speakers_all.csv")

выводим и смотрим на данные:

In [3]:
dataframe.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True,NaN,NaN,NaN
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True,NaN,NaN,NaN
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True,NaN,NaN,NaN
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True,NaN,NaN,NaN
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True,NaN,NaN,NaN


### Отобрать записи, соответствующие странам с минимум 30 респонеднтами в выборке

Сделаем это следующим образом: сначала отбросим тех, у кого нет файла

In [4]:
dataframe = dataframe[dataframe['file_missing?'] == False]

теперь сгруппируем по странам и выведем с минимум 30 респонеднтами в выборке

In [5]:
counts = dataframe.groupby('country').agg(['count'])
index_countries = counts[counts['speakerid']['count']>30].index
agg_dataframe = dataframe[dataframe.country.isin(index_countries)]


Выводим таблицу

In [6]:
agg_dataframe.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,Unnamed: 9,Unnamed: 10,Unnamed: 11
50,20.0,5.0,"addis ababa, ethiopia",amharic1,amharic,female,6,ethiopia,False,NaN,NaN,NaN
51,29.0,19.0,"gonder, ethiopia",amharic10,amharic,female,998,ethiopia,False,NaN,NaN,NaN
52,24.0,17.0,"addis ababa, ethiopia",amharic11,amharic,female,1129,ethiopia,False,NaN,NaN,NaN
53,26.0,15.0,"addis ababa, ethiopia",amharic12,amharic,female,1130,ethiopia,False,NaN,NaN,NaN
54,28.0,6.0,"addis ababa, ethiopia",amharic13,amharic,female,1131,ethiopia,False,NaN,NaN,NaN


Добавим поле для zrc и удалим лишнии поля

In [7]:
agg_dataframe['zcr'] = 0

del agg_dataframe['file_missing?']
del agg_dataframe['Unnamed: 9']
del agg_dataframe['Unnamed: 10']
del agg_dataframe['Unnamed: 11']
del agg_dataframe['speakerid']
del agg_dataframe['age_onset']
del agg_dataframe['country']
del agg_dataframe['birthplace']

agg_dataframe.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,age,filename,native_language,sex,zcr
50,20.0,amharic1,amharic,female,0
51,29.0,amharic10,amharic,female,0
52,24.0,amharic11,amharic,female,0
53,26.0,amharic12,amharic,female,0
54,28.0,amharic13,amharic,female,0


### Считаем zrc

In [8]:
# values = agg_dataframe.values

# for obj in tqdm(values):
#     a , sr = librosa.load('recordings/'+obj[1]+'.mp3')
#     zero_crossings = librosa.zero_crossings(a, pad=False)
#     obj[-1] = sum(zero_crossings)
# agg_dataframe = pd.DataFrame(values, columns=agg_dataframe.columns)
# del agg_dataframe['filename']
# agg_dataframe.to_csv('full_data.csv', index=False)

Закомментируем эту часть, так как она занимает много времени и будет открывать уже сохраненную таблицу

### ANOVA анализ

In [9]:
dataframe = pd.read_csv('full_data.csv')

In [10]:
dataframe.head()

,age,nl,sex,zcr
0,20.0,amharic,female,67648
1,29.0,amharic,female,73605
2,24.0,amharic,female,80432
3,26.0,amharic,female,64450
4,28.0,amharic,female,103724


Так как количество уникальных пар (язык, возраст) больше количества объектов в выборке, то если мы оставим в таком виде, то задача будет не корректной.

Введем возрастные группы, а именно разобъем все объекты по возрасту с шагом в 11 лет

Такой прием дает нам возможность использовать ANOVA анализ

In [11]:
X = dataframe.values
new_age = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), 11)
res = (X[:, 0] - new_age.reshape([-1,1])).T

new_index = np.argmin(np.abs(res), axis = -1)

dataframe['age'] = new_age[new_index]

In [12]:
dataframe.head()

,age,nl,sex,zcr
0,17.0,amharic,female,67648
1,28.0,amharic,female,73605
2,28.0,amharic,female,80432
3,28.0,amharic,female,64450
4,28.0,amharic,female,103724


### Теперь перейдем непосредственно к ANOVA

In [16]:
lm = ols('zcr ~ C(sex)*C(age)*C(nl)', data = dataframe).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(sex),1.0,1.777982e+10,1.777982e+10,26.627208,3.074567e-07
C(age),8.0,1.554634e+10,1.943293e+09,2.910292,3.301338e-03
C(nl),55.0,2.644670e+11,4.808490e+09,7.201236,9.699259e-42
C(sex):C(age),8.0,3.822639e+09,4.778298e+08,0.715602,6.779228e-01
C(sex):C(nl),55.0,4.680000e+10,8.509090e+08,1.274329,9.130925e-02
C(age):C(nl),440.0,3.089407e+11,7.021381e+08,1.051528,2.693417e-01
C(sex):C(age):C(nl),440.0,3.113391e+11,7.075889e+08,1.059691,2.393622e-01
Residual,848.0,5.662361e+11,6.677312e+08,NaN,NaN


In [34]:
multipletests(table.values[:, -1][:-1], alpha=0.15, method='holm')[0]

array([ True,  True,  True,  True])

Межфакторное взаемодействие всех трех незначимо, поэтому используем только попарные.

In [30]:
lm = ols('zcr ~ C(sex)*C(nl)+C(age)*C(nl)+C(age)*C(sex)', data = dataframe).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(sex),1.0,1.777982e+10,1.777982e+10,26.017268,4.147958e-07
C(nl),55.0,2.567531e+11,4.668237e+09,6.831047,2.242574e-39
C(age),8.0,2.326026e+10,2.907532e+09,4.254602,4.989679e-05
C(sex):C(nl),55.0,8.501600e+10,1.545745e+09,2.261894,1.031580e-06
C(age):C(nl),440.0,2.847537e+11,6.471676e+08,0.947003,7.414013e-01
C(age):C(sex),8.0,3.858149e+09,4.822686e+08,0.705705,6.867305e-01
Residual,877.0,5.993289e+11,6.833853e+08,NaN,NaN


In [32]:
multipletests(table.values[:, -1][:-1], alpha=0.15, method='holm')[0]

array([ True,  True,  True,  True, False, False])

Межфакторныя взаемодействия между парами (age, nl) и (age, sex) не значимы. 

In [33]:
lm = ols('zcr ~ C(sex)*C(nl)+C(age)', data = dataframe).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(sex),1.0,1.777982e+10,1.777982e+10,24.418176,9.152531e-07
C(nl),55.0,2.567531e+11,4.668237e+09,6.411193,6.576839e-37
C(age),8.0,2.326026e+10,2.907532e+09,3.993102,1.134734e-04
C(sex):C(nl),55.0,1.091337e+11,1.984249e+09,2.725098,9.141371e-10
Residual,955.0,6.953725e+11,7.281387e+08,NaN,NaN


Межфакторное взаемодействие между sex и nl значимо, поэтому поделим по sex

In [36]:
lm = ols('zcr ~ C(nl)+C(age)', data = dataframe.loc[dataframe['sex'] == 'female']).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(nl),42.0,1.998651e+11,4.758693e+09,6.753753,1.089172e-27
C(age),7.0,1.328823e+10,1.898319e+09,2.694181,9.612484e-03
Residual,454.0,3.198883e+11,7.045997e+08,NaN,NaN


In [38]:
multipletests(table.values[:, -1][:-1], alpha=0.15, method='holm')[0]

array([ True,  True])

In [37]:
lm = ols('zcr ~ C(nl)+C(age)', data = dataframe.loc[dataframe['sex'] == 'male']).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(nl),38.0,6.978083e+10,1.836338e+09,2.433295,0.000008
C(age),8.0,1.346716e+10,1.683395e+09,2.230634,0.024125
Residual,494.0,3.728076e+11,7.546712e+08,NaN,NaN


In [39]:
multipletests(table.values[:, -1][:-1], alpha=0.15, method='holm')[0]

array([ True,  True])

### Вывод
Полученные результаты показывают, что межфакторное взаемодействие в данной задаче важно только между полом и возрастом испытумых. Также было получено, что трехфакторное взаемодействие не нужно.